# Energy Management in Building Project 

## Introduction

L'objectif de ce projet est ...

## General presentation of the building 

### Description of the studio
We are working on a studio of 32 $m^{2}$ in a builnding

![studio](plan_studio.png)